In [1]:
import os
import sys
import wandb
import torch
import datetime
import collections
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml

os.environ['WANDB_NOTEBOOK_NAME'] = 'train_classification.ipynb'

HPARAM_FILE = 'hparams/class/twin_tcn.yaml'

In [2]:
time_stamp = datetime.datetime.now().strftime('%Y-%m-%d+%H-%M-%S')
print(f'Experiment Time Stamp: {time_stamp}')

# Overwrite hparams
argv = [HPARAM_FILE, '--time_stamp', time_stamp]
argv += ['--use_wandb', 'true']
argv += ['--n_mismatch', '4']
argv += ['--experiment', 'env_tcn_bs4_lr1e-3_ep200']

hparam_file, run_opts, overrides = sb.parse_arguments(argv)

with open(HPARAM_FILE) as f:
    hparams = load_hyperpyyaml(f, overrides)
    
run_opts['auto_mix_prec'] = hparams['mix_prec'] # False

if hparams['use_wandb']:
    hparams['logger'] = hparams['wandb_logger']()
    
# sb.utils.distributed.ddp_init_group(run_opts)

Experiment Time Stamp: 2023-11-26+17-42-06


wandb: Currently logged in as: xj2289 (xj-audio). Use `wandb login --relogin` to force relogin


In [ ]:
from tqdm import tqdm
train_windows = hparams['train_windows']
for windows in tqdm(train_windows, total=len(train_windows)):
    pass

In [3]:
class Classifier(sb.core.Brain):

    def compute_forward(self, eeg, speech, stage):
        pred = self.modules.twin_model(eeg, speech)
        
        return pred
    
    def compute_objectives(self, pred, label, stage):
        B = label.shape[0]
        
        loss = self.hparams.loss_fn(pred, label)
        est_label = torch.argmax(pred, dim=-1)
        acc = sum(est_label==label) / B
        
        self.loss_stat['loss'] += float(loss) * B
        self.loss_stat['acc'] += float(acc) * B
        self.count += B
    
        return loss

    def fit_batch(self, batch):
        eeg = batch['eeg'].to(self.device)
        speech = batch[self.hparams.feature].to(self.device)
        
        # Select one EEG as the target. Keep all M speeches as candidates.
        B, M, C, T = eeg.shape
        label = torch.randint(0, M, (B,), dtype=torch.long).to(self.device)
        eeg = eeg[torch.arange(B), label]
        
        # Forward
        pred = self.compute_forward(eeg, speech, sb.Stage.TRAIN)
        loss = self.compute_objectives(pred, label, sb.Stage.TRAIN)
        loss.backward()
        
        if self.check_gradients(loss):
            self.optimizer.step()
        self.optimizer.zero_grad()
            
        return loss.detach().cpu()
    
    def evaluate_batch(self, batch, stage):
        eeg = batch['eeg'].to(self.device)
        speech = batch[self.hparams.feature].to(self.device)

        # Select one EEG as the target. Keep all M speeches as candidates.
        B, M, C, T = eeg.shape
        label = torch.zeros(B, dtype=torch.long).to(self.device)
        eeg = eeg[torch.arange(B), label]
        
        # Forward
        with torch.no_grad():
            pred = self.compute_forward(eeg, speech, stage)
            loss = self.compute_objectives(pred, label, stage)
            
        return loss.detach().cpu()
        
    def on_stage_start(self, stage, epoch=None):
        super().on_stage_start(stage, epoch)
        self.count = 0
        self.loss_stat = {
            'loss': 0,
            'acc': 0,
        }

    def on_stage_end(self, stage, stage_loss, epoch=None):
        for loss_key in self.loss_stat:
            self.loss_stat[loss_key] /= self.count
        
        if stage == sb.Stage.TRAIN:
            stage_stats = {'train_'+key: round(float(value), 4) for key, value in self.loss_stat.items()}
            stage_stats['epoch'] = epoch
    
        elif stage == sb.Stage.VALID:
            stage_stats = {'valid_'+key: round(float(value), 4) for key, value in self.loss_stat.items()}
            stage_stats['epoch'] = epoch

        if self.hparams.use_wandb:
            self.hparams.logger.run.log(
                data=stage_stats,
            )
                        
        print(f'Epoch {epoch}: ', stage, stage_stats)
                        
    

In [4]:
brain = Classifier(
    modules=hparams['modules'],
    opt_class=hparams['optimizer'],
    hparams=hparams,
    run_opts=run_opts,
    checkpointer=hparams['checkpointer'],
)

In [ ]:
brain.fit(
    epoch_counter=brain.hparams.epoch_counter,
    train_set=hparams['train_windows'],
    valid_set=hparams['valid_windows'],
    train_loader_kwargs=hparams['train_loader_opts'],
    valid_loader_kwargs=hparams['valid_loader_opts'],
)

100%|██████████| 166/166 [00:13<00:00, 12.54it/s, train_loss=1.6] 


Epoch 1:  Stage.TRAIN {'train_loss': 1.6024, 'train_acc': 0.2455, 'epoch': 1}


100%|██████████| 167/167 [00:06<00:00, 25.22it/s]


Epoch 1:  Stage.VALID {'valid_loss': 1.5787, 'valid_acc': 0.2898, 'epoch': 1}


100%|██████████| 166/166 [00:12<00:00, 13.42it/s, train_loss=1.54]


...

100%|██████████| 167/167 [00:06<00:00, 24.97it/s]


Epoch 12:  Stage.VALID {'valid_loss': 1.4128, 'valid_acc': 0.4009, 'epoch': 12}


100%|██████████| 166/166 [00:12<00:00, 13.41it/s, train_loss=1.4] 


Epoch 13:  Stage.TRAIN {'train_loss': 1.4027, 'train_acc': 0.4337, 'epoch': 13}


  0%|          | 0/167 [00:00<?, ?it/s]

In [ ]:
wandb.finish()